# Digital Filters in Audio

Author: J.D. Cardenas-Cartagena


In the last lecture, we discussed how FIR filters can modify the amplitude and phase of an input signal using their magnitude and angle, respectively. In this tutorial, we will design a basic audio equalizer using FIR filters. Key Concept for today:

**Decibel (dB) and Level:**  It is one-tenth of a Bel, a relative unit used to express a ratio. In audio, it represents the sound level or volume. In this tutorial we will use the following reference:
$$
Level_{dB}(A_{in}, A_{out}) = 20 \log_{10} \left( \frac{A_{out}}{A_{in}} \right)
$$
For example, an amplifier with a +10 dB gain means that the output signal is 10 times more powerful than the input signal, and the amplitude increases by a factor of approximately 3.16.

**Equalizer:** It adjusts the amplitude of different frequency components in an audio signal by boosting or cutting specific frequency bands.

## FIR Filter Design by Windowing

We can design FIR filters by specifying the desired frequency response and then applying a window function to control the filter's behavior. The basic idea of this design process begins by specifying a sinc function for low-pass, high-pass, or band-pass filters.

<figure>
<center>
<img src="img/low-pass-filter.png" />
<figcaption>Figure 1: Low-pass FIR Filter. 
mage taken from Hardware Acceleration of FIR Filter Implementation on ZYNQ SoC - Scientific Figure on ResearchGate. </figcaption>
</center>
</figure>

 The sinc function behaves like the ideal impulse response of an FIR filter, as it corresponds to a rectangular function in the frequency domain. Since the ideal impulse response is typically infinite, we must truncate it. This truncation introduces negative effects, such as oscillations in the frequency response and abrupt changes in the time-domain signal. To mitigate these issues, we use window functions, like Hamming, to smooth the response and enhance the filter's performance.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import firwin, freqz, lfilter
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import soundfile as sf
import librosa as lr
from IPython.display import Audio

ModuleNotFoundError: No module named 'plotly'

In [ ]:
# Filter parameters
num_taps = 101  # Same as your plot
normalized_cutoff = 0.2  # Normalized frequency

# Generate ideal impulse response
n = np.arange(num_taps)
center = (num_taps - 1) / 2

# Original incorrect calculation:
# h_ideal = np.sinc(2 * normalized_cutoff * (n - center))

# Corrected calculation:
h_ideal = np.sinc(normalized_cutoff * (n - center)) * normalized_cutoff * 2

# Calculate frequency response
w, H = freqz(h_ideal, worN=8000)

# Create subplots
plt.figure(figsize=(15, 10))

# Plot impulse response
plt.subplot(3, 1, 1)
plt.stem(n, h_ideal)
plt.title('Corrected Impulse Response (Sinc Function)')
plt.xlabel('Tap number')
plt.ylabel('Amplitude')
plt.grid(True)

# Plot magnitude response
plt.subplot(3, 1, 2)
plt.plot(w/np.pi, np.abs(H))
plt.axvline(normalized_cutoff, color='r', linestyle='--', label='Cutoff Frequency')
plt.axvline(1, color='g', linestyle=':', label='Nyquist Frequency')
plt.title('Magnitude Response')
plt.xlabel('Normalized Frequency (×π rad/sample)')
plt.ylabel('Magnitude')
plt.grid(True)
plt.legend()

# Plot phase response
plt.subplot(3, 1, 3)
phase = np.angle(H)
plt.plot(w/np.pi, phase)
plt.title('Phase Response')
plt.xlabel('Normalized Frequency (×π rad/sample)')
plt.ylabel('Phase (radians)')
plt.grid(True)

# Add vertical line at the nyquist frequency
plt.axvline(1, color='g', linestyle=':', label='Nyquist Frequency')
plt.legend()

plt.tight_layout()
plt.show()

## Behaviour of Different Window Functions

In [ ]:
# Filter specifications
num_taps = 101
normalized_cutoff = 0.2

# Create time index
n = np.arange(num_taps)
center = (num_taps - 1) / 2

# Generate ideal impulse response (sinc)
h_ideal = np.sinc(normalized_cutoff * (n - center)) * normalized_cutoff * 2

# Define different windows
rectangular = np.ones(num_taps)
hamming = np.hamming(num_taps)
hanning = np.hanning(num_taps)
blackman = np.blackman(num_taps)
kaiser = np.kaiser(num_taps, beta=8)

# Apply windows to create different filters
h_rect = h_ideal * rectangular
h_hamming = h_ideal * hamming
h_hanning = h_ideal * hanning
h_blackman = h_ideal * blackman
h_kaiser = h_ideal * kaiser

# Calculate frequency responses
w, H_rect = freqz(h_rect)
_, H_hamming = freqz(h_hamming)
_, H_hanning = freqz(h_hanning)
_, H_blackman = freqz(h_blackman)
_, H_kaiser = freqz(h_kaiser)

# Create interactive subplot
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Window Functions', 'Frequency Response'),
    vertical_spacing=0.15
)

# Add window function traces
windows = {
    'Rectangular': rectangular,
    'Hamming': hamming,
    'Hanning': hanning,
    'Blackman': blackman,
    'Kaiser': kaiser
}

for name, window in windows.items():
    fig.add_trace(
        go.Scatter(x=n, y=window, name=name, mode='lines'),
        row=1, col=1
    )

# Add frequency response traces
responses = {
    'Rectangular': H_rect,
    'Hamming': H_hamming,
    'Hanning': H_hanning,
    'Blackman': H_blackman,
    'Kaiser': H_kaiser
}

for name, response in responses.items():
    fig.add_trace(
        go.Scatter(
            x=w/np.pi,
            y=20 * np.log10(np.abs(response)),
            name=name,
            mode='lines'
        ),
        row=2, col=1
    )

# Add cutoff frequency line
fig.add_vline(
    x=normalized_cutoff,
    line_dash="dash",
    line_color="red",
    annotation_text="Cutoff Frequency",
    row=2, col=1
)

# Update layout
fig.update_layout(
    title_text="Interactive FIR Filter Window Comparison",
    height=800,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

# Update axes labels
fig.update_xaxes(title_text="Sample", row=1, col=1)
fig.update_yaxes(title_text="Amplitude", row=1, col=1)
fig.update_xaxes(title_text="Normalized Frequency (×π rad/sample)", row=2, col=1)
fig.update_yaxes(title_text="Magnitude (dB)", row=2, col=1)

# Update y-axis range for frequency response
fig.update_yaxes(range=[-100, 10], row=2, col=1)

# Add grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

# Show the plot
fig.show()

## Let us Build an Equalizer with FIR Filters

We will modify the audio signal by dividing the audio spectrum into the following frequency bands:
* Sub Bass: 20-60 Hz
* Bass: 60-250 Hz
* Low Mids: 250-500 Hz
* Mids: 500-2000 Hz
* High Mids: 2000-4000 Hz
* Presence: 4000-6000 Hz
* Brilliance: 6000-20000 Hz
And adjust the gain of each band independently. 

Note: We can use the scipy.signal.firwin() function to design the FIR filter.

In [ ]:
# Parameters for the bandpass filter
fs = 44100      # Sampling frequency (Hz)
lowcut = 300    # Lower cutoff frequency (Hz)
highcut = 3000  # Higher cutoff frequency (Hz)
numtaps = 1001  # Number of taps - make it odd for Type I filter

# Design the bandpass filter
b = firwin(numtaps, [lowcut, highcut], fs=fs, pass_zero=False, window='hamming')

# Calculate frequency response
frequencies = np.linspace(0, fs/2, 4000)  # Create frequency points up to Nyquist
w, h = freqz(b, worN=frequencies, fs=fs)

# Plot the frequency response
plt.figure(figsize=(10, 6))
plt.plot(w, 20 * np.log10(np.abs(h)))
plt.grid(True)
plt.title('Bandpass Filter Frequency Response')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')

# Add markers for cutoff frequencies
plt.axvline(lowcut, color='r', linestyle='--', alpha=0.5, label=f'Lowcut ({lowcut} Hz)')
plt.axvline(highcut, color='g', linestyle='--', alpha=0.5, label=f'Highcut ({highcut} Hz)')
plt.axhline(-3, color='y', linestyle=':', alpha=0.5, label='-3 dB point')

plt.xscale('log')
plt.xlim(20, fs/2)
plt.ylim(-80, 5)
plt.legend()
plt.show()

# Print the filter coefficients
print("Filter coefficients:", b)


In [ ]:
### Step 1: Define the Frequency Bands and Gain Settings
# Frequency bands in Hz
bands = {
    'sub_bass': (20, 60),
    'bass': (60, 250),
    'low_mids': (250, 500),
    'mids': (500, 2000),
    'high_mids': (2000, 4000),
    'presence': (4000, 6000),
    'brilliance': (6000, 20000)
}

# Gain settings for each band in dB (students can experiment with these values)
gain_settings = {
    'sub_bass': 6,    # Boost
    'bass': 10,        # Boost
    'low_mids': 0,    # Neutral
    'mids': -10,       # Cut
    'high_mids': 0,   # Neutral
    'presence': -10,   # Cut
    'brilliance': 15   # Boost
}

# Sampling frequency (common audio sample rate)
fs = 44100  # 44.1 kHz

# Filter order (higher values create sharper filters but increase computation time)
numtaps = 1001

### Step 2: Design Band-Pass FIR Filters
# Function to design a band-pass FIR filter for a specific frequency range
def design_filter(lowcut, highcut, fs, numtaps):
    return firwin(numtaps, [lowcut, highcut], fs=fs, pass_zero=False)

# Dictionary to store filters for each band
filters = {}
for band, (lowcut, highcut) in bands.items():
    filters[band] = design_filter(lowcut, highcut, fs, numtaps)

# Visualize the frequency response of each filter interactively
fig = go.Figure()
for band, b in filters.items():
    w, h = freqz(b, worN=8000)
    fig.add_trace(go.Scatter(x=w * fs / (2 * np.pi), y=20 * np.log10(abs(h)), mode='lines', name=band))

# Add horizontal lines for frequency bands
for band, (lowcut, highcut) in bands.items():
    fig.add_shape(type="line", 
                  x0=lowcut, y0=-50, x1=lowcut, y1=0,
                  line=dict(color="gray", dash="dot"))
    fig.add_shape(type="line", 
                  x0=highcut, y0=-50, x1=highcut, y1=0,
                  line=dict(color="gray", dash="dot"))
    fig.add_trace(go.Scatter(
        x=[(lowcut + highcut) / 2], 
        y=[-40], 
        text=[band],
        mode="text",
        showlegend=False
    ))

fig.update_layout(
    title="Frequency Response of Band-Pass Filters",
    xaxis_title="Frequency (Hz)",
    yaxis_title="Amplitude (dB)",
    legend_title="Frequency Bands",
    template="plotly_white"
)
fig.show()

### Step 3: Load an Audio File
audio_file = 'audio_input.mp3'
# Load the audio file (mono)
signal, sr = lr.load(audio_file, sr=fs)

# Plot the original signal interactively
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(signal)), y=signal, mode='lines', name='Original Signal'))
fig.update_layout(
    title="Original Audio Signal",
    xaxis_title="Samples",
    yaxis_title="Amplitude",
    template="plotly_white"
)
fig.show()

### Step 4: Apply the Equalizer
# Apply each filter and adjust the gain
equalized_signal = np.zeros_like(signal)

for band, b in filters.items():
    # Filter the signal for the current band
    filtered_signal = lfilter(b, 1.0, signal)
    
    # Apply gain (convert dB to linear scale)
    gain = 10 ** (gain_settings[band] / 20)
    filtered_signal *= gain
    
    # Accumulate the filtered signals
    equalized_signal += filtered_signal

# Plot the equalized signal interactively
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(equalized_signal)), y=equalized_signal, mode='lines', name='Equalized Signal'))
fig.update_layout(
    title="Equalized Audio Signal",
    xaxis_title="Samples",
    yaxis_title="Amplitude",
    template="plotly_white"
)
fig.show()

### Step 5: Save the Equalized Audio
# Save the equalized audio to a new file
output_file = 'equalized_audio.wav'
sf.write(output_file, equalized_signal, sr)

print(f"Equalized audio saved as {output_file}")

### Step 6: Analyze the Frequency Response of the original and Equalized Signal
# Compute the frequency response of the original signal
w_orig, h_orig = freqz(signal, worN=8000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=w_orig * fs / (2 * np.pi), y=20 * np.log10(abs(h_orig)), mode='lines', name='Original Signal'))

# Compute the frequency response of the equalized signal
w_eq, h_eq = freqz(equalized_signal, worN=8000)
fig.add_trace(go.Scatter(x=w_eq * fs / (2 * np.pi), y=20 * np.log10(abs(h_eq)), mode='lines', name='Equalized Signal'))

# Add horizontal lines for frequency bands
for band, (lowcut, highcut) in bands.items():
    fig.add_shape(type="line", 
                  x0=lowcut, y0=-50, x1=lowcut, y1=0,
                  line=dict(color="gray", dash="dot"))
    fig.add_shape(type="line", 
                  x0=highcut, y0=-50, x1=highcut, y1=0,
                  line=dict(color="gray", dash="dot"))
    fig.add_trace(go.Scatter(
        x=[(lowcut + highcut) / 2], 
        y=[-40], 
        text=[band],
        mode="text",
        showlegend=False
    ))

fig.update_layout(
    title="Frequency Response Comparison",
    xaxis_title="Frequency (Hz)",
    yaxis_title="Amplitude (dB)",
    legend_title="Signals",
    template="plotly_white"
)
fig.show()



Note: The example audio file is [MIT Symphony Orchestra - La Traviata, Brindisi (Verdi)](https://freemusicarchive.org/music/MIT_Symphony_Orchestra/An_Opera_Evening)

Further reading:
Chapter 7 in P. Prandoni and M. Vetterli, Signal processing for communications. in Communication and information sciences. Lausanne (Suisse) Boca Raton (États-Unis): EPFL Press CRC Press, 2008.